In [56]:
#import dependencies

import pandas as pd

In [57]:
#read in csvs

electiondata = pd.read_csv("1976-2016-president.csv")
rate1 = pd.read_csv("1980-2014 November General Election - Turnout Rates.csv")
rate2 = pd.read_csv("2016 November General Election - Turnout Rates.csv")
electoralvotes = pd.read_csv("Available Electoral votes .csv")

In [58]:
####pull party info for normalization


#parties = electiondata[["year","state","candidate", "party"]]
#parties.drop_duplicates(inplace=True)


#parties.to_csv("allparties.csv", index=False)
#parties

In [59]:
#clean up voting data 

#file one
rate1.rename(columns={"Unnamed: 0": "year", "Denominators" : "Eligible_voting_pop"}, inplace= True)
voting_data_1 = rate1[["year", "State", "Eligible_voting_pop"]]
voting_data_1 = voting_data_1.drop(voting_data_1.index[0])

#file two
rate2.rename(columns={"Denominators" : "Eligible_voting_pop"}, inplace= True)
rate2["year"] = '2016'
voting_data_2 = rate2[["year", "State", "Eligible_voting_pop"]]
voting_data_2 = voting_data_2.drop(voting_data_2.index[0])

In [60]:
#merge dataframes

total_voting = voting_data_2.append(voting_data_1)

In [61]:
#check dtypes on dataframes

#total_voting.dtypes
    #year is object

#electiondata.dtypes
    #year is int64

In [62]:
#convert year to int for merge

total_voting['year'] = total_voting.year.astype(str).astype(int)

In [63]:
#merge dataframes 

merged_df = electiondata.merge(total_voting, how='left', left_on=['state','year'], right_on=["State","year"])


In [64]:
#merge electoral college votes dataframe and drop columns

working_df = merged_df.merge(electoralvotes, how='left', left_on=['state','year'], right_on=["State","Year"])
final_working_df = working_df.drop(['state_po', 'state_fips', "state_cen", "state_ic", "office", "version", "notes", "State ID", "State_x","State_y", "Year", "writein"], axis=1)


In [65]:
#create key for merge and man
final_working_df["new"] = final_working_df["candidate"] + final_working_df["year"].astype(str)

final_working_df.replace("Mitt, Romney", "Romney, Mitt", inplace=True)
final_working_df.replace("Hedges, Jim", "Hedges, James", inplace=True)
final_working_df.replace("Haylard, Helen", "Halyard, Helen", inplace=True)
final_working_df.replace("Stevens, Thomas Robert ""Tom""", "Stevens, Thomas R.", inplace=True)
final_working_df.replace("White, Jerry", "White, Jerome ""Jerry""", inplace=True)


final_working_df["candidate"] = final_working_df['candidate'].fillna("unknown")
final_working_df["party"] = final_working_df['party'].fillna("unknown")
final_working_df["Eligible_voting_pop"] = final_working_df["Eligible_voting_pop"].fillna(.00000000000000000000000000000000000000000000000001)

final_working_df["Eligible_voting_pop"] = final_working_df["Eligible_voting_pop"].replace({'\$': '', ',': ''}, regex=True)
final_working_df["Eligible_voting_pop"] = final_working_df["Eligible_voting_pop"].astype(str).astype(float).round(0)

In [66]:
#read in manipulated party csv data

allparties = pd.read_csv("allparties.csv")

#merge all parties with final working for normalized party info 
final_working_df = final_working_df.merge(allparties,how= "left", left_on= ["candidate", "state", "year"], right_on=["candidate", "state", "year"])


In [67]:
###TROUBLESHOOT CELL IGNORE!!!!

#final_working_df = final_working_df.loc[final_working_df["year"] == 2000]
#final_working_df = final_working_df.loc[final_working_df["state"] == "Oregon"]
#final_working_df = final_working_df.loc[final_working_df["candidate"] == 'Other']

#final_working_df

In [68]:
# fill join column for search and replace

final_working_df["party_y"] = final_working_df['party_y'].fillna(0)

for row in final_working_df["party_y"]:
    final_working_df.loc[final_working_df["party_y"] != 0, ["party_x"]] = final_working_df["party_y"]

In [69]:
#rename and drop

final_working_df.rename(columns={"party_x" : "party"}, inplace=True)
final_working_df = final_working_df.drop("party_y", axis= 1)
final_working_df = final_working_df.drop_duplicates()

In [70]:
final_working_df = final_working_df.groupby(["year", "state", "candidate", "party", "totalvotes", "Eligible_voting_pop", "electoralvotes", "new"]).sum().reset_index()

In [71]:
###TROUBLESHOOT CELL IGNORE!!!!

#final_working_df = final_working_df.loc[final_working_df["year"] == 2016]
#final_working_df = final_working_df.loc[final_working_df["state"] == "Oregon"]
#final_working_df

In [72]:
#find top candidates by popular vote

find_top_candidates_df = final_working_df[["year","candidate", "candidatevotes"]]

find_top_candidates_df = find_top_candidates_df.groupby(["candidate", "year"]).sum().reset_index()

find_top_candidates_df= find_top_candidates_df.sort_values(by=["year", "candidatevotes"], ascending=False).groupby("year").head(2)

top_candidates_df = pd.DataFrame(find_top_candidates_df)

#create key for merge
top_candidates_df["new"] = top_candidates_df["candidate"] + top_candidates_df["year"].astype(str)

#create dataframe with only key for merge
#loop through to deduplicate 
only_winners = top_candidates_df[["new"]]

toppy = []
for row in only_winners["new"]:
    if row not in toppy:
        toppy.append(row)

toppy=pd.DataFrame(toppy)

In [73]:
#merge to create column for flag

final_working_df = final_working_df.merge(toppy, how="left", left_on= "new", right_on= 0)

#fill na in new columnt with 0
final_working_df[0] = final_working_df[0].fillna(0)

In [74]:
#using flag, change candidate name to other if not in top 2

final_working_df.loc[final_working_df[0] == 0, ["candidate"]] = 'Other'

#drop flag column
final_working_df = final_working_df.drop([0], axis=1)

In [75]:
find_top_candidates_state_df = final_working_df[["year","state","candidate", "candidatevotes", "new"]]

In [76]:
#sum candidate votes

find_top_candidates_state_df = find_top_candidates_state_df.groupby(["candidate", "year", "state"]).sum().reset_index()

#sort to get top candidate
find_top_candidates_state_df= find_top_candidates_state_df.sort_values(by=[ "year","candidatevotes", "state"], ascending=False)
find_top_candidates_state_df = find_top_candidates_state_df.sort_values(["year", "state"]).groupby(["state","year"]).head(1)

#create key for merge
find_top_candidates_state_df["new"] = find_top_candidates_state_df["candidate"] + find_top_candidates_state_df["year"].astype(str)

In [77]:
#merge dataframed for the state won flag

state_winner_added = final_working_df.merge(find_top_candidates_state_df, how="left", on= ["new", "state"], suffixes=('', 'drop'))

#fill na in flag column for loc and change
state_winner_added["candidatedrop"] = state_winner_added["candidatedrop"].fillna(0)

#loc and change values
state_winner_added.loc[state_winner_added["candidatedrop"] != 0, ["candidatedrop"]] = '1'
state_winner_added.loc[state_winner_added["candidatedrop"] == 0, ["candidatedrop"]] = '0'

#math for % won
state_winner_added["percent_won"] = state_winner_added["candidatevotes"] / state_winner_added["totalvotes"]

#drop and rename columns
state_winner_added = state_winner_added.drop(["yeardrop",  "candidatevotesdrop", "new"], axis=1).rename(columns={"candidatedrop": "state_won_flag"})

final_df = state_winner_added

In [78]:
#electoral college df
electoral_college = final_df[["year", "state", "candidate","electoralvotes", "state_won_flag"]]
electoral_college = electoral_college.loc[electoral_college["state_won_flag"] == '1']

electoral_college = electoral_college.drop(["state","state_won_flag" ], axis=1)
electoral_college = electoral_college.groupby(["year", "candidate"]).sum().reset_index()

electoral_college["new"] = electoral_college["candidate"] + electoral_college["year"].astype(str)

electoral_college_win = electoral_college.groupby("year").head(2)




In [79]:
#start of year level dataset

year_df = final_df.groupby(["year", "candidate"]).sum().reset_index().sort_values(by= ["year", "candidatevotes"], ascending=False).groupby("year").head(1)
popular_vote = year_df[["year", "candidate", "candidatevotes"]]
popular_vote["new"] = popular_vote["candidate"] + popular_vote["year"].astype(str)



year_df = final_df.groupby(["year", "candidate"]).sum().reset_index().sort_values(by= ["year", "candidatevotes"], ascending=False).groupby("year").head(3)

vote = year_df[["year", "candidate", "candidatevotes"]]
vote["new"] = vote["candidate"] + vote["year"].astype(str)

vote = vote.merge(popular_vote, how="left", left_on="new", right_on="new", suffixes=('', 'drop'))
vote = vote.merge(electoral_college, how="left", left_on="new", right_on="new", suffixes=('', 'drop1'))

In [80]:
vote = vote.drop(["new", "yeardrop", "yeardrop1", "candidatevotesdrop", "candidatedrop1"], axis=1)
vote["electoralvotes"] = vote["electoralvotes"].fillna(0)
vote["candidatedrop"] = vote["candidatedrop"].fillna(0)

vote.loc[vote["candidatedrop"] != 0, ["candidatedrop"]] = '1'
#vote.loc[vote["candidatedrop"] == 0, ["candidatedrop"]] = '0'

vote.rename(columns={"candidatedrop": "won_pop_vote_flag"}, inplace=True)

In [81]:
turnout_votes = final_working_df[["year", "totalvotes"]]
turnout_eligible = final_working_df[["year", "Eligible_voting_pop"]]

turnout_votes.drop_duplicates(inplace=True)
#turnout_votes

turnout_eligible.drop_duplicates(inplace=True)
#turnout_eligible

turnout_votes = turnout_votes.groupby("year").sum()
#turnout_votes

turnout_eligible = turnout_eligible.groupby("year").sum().reset_index()

turnout = turnout_votes.merge(turnout_eligible, how = "left", on="year")
turnout

turnout["perc"] = turnout["totalvotes"]/ turnout["Eligible_voting_pop"].round(2)

In [83]:
#read all data to csvs

final_df.to_csv("../final_data/state_level_data.csv")
vote.to_csv("../final_data/year_level_data.csv")
final_df.to_csv("../final_data/turnout_data.csv")